In [43]:
# Importació de llibreries bàsiques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast  # Per processar els embeddings
from pathlib import Path

# Preprocessament i Modelització
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# Càrrega de dades
data_dir = Path('.')
train_path = data_dir / 'train.csv'
test_path = data_dir / 'test.csv'
sample_path = data_dir / 'sample_submission.csv'

try:
    train_df = pd.read_csv(train_path, sep=';')
    test_df = pd.read_csv(test_path, sep=';')
    sample_sub = pd.read_csv(sample_path, sep=',')
except FileNotFoundError:
    print("Error: Assegura't que els fitxers 'train.csv', 'test.csv' i 'sample_submission.csv' estan al mateix directori.")

print("Dades carregades:")
print(f"Train:   {train_df.shape}")
print(f"Test:    {test_df.shape}")
print(f"Sample:  {sample_sub.shape}")

Dades carregades:
Train:   (95339, 33)
Test:    (2250, 33)
Sample:  (2250, 2)


In [52]:
# --- Paso 1: Tarea 2 (Inspección Rápida) ---

print("--- Vistazo a train_df (head) ---")
print(train_df.head())
print("\n")

print("--- Información de train_df (.info()) ---")
# Esto nos dirá los tipos de datos (Dtypes) y los nulos
train_df.info()
print("\n")

print("--- Información de test_df (.info()) ---")
test_df.info()
print("\n")

# --- Paso 1: Tarea 3 (Validar Hipótesis de Agregación) ---

print("--- Conteo de filas por 'id' en train_df ---")
# Si vemos números mayores a 1, significa que hay varias filas por id (semanal)
print(train_df['ID'].value_counts().head(10))
print("\n")

print("--- Conteo de filas por 'id' en test_df ---")
# Aquí deberíamos ver que todos los valores son '1'
print(test_df['ID'].value_counts().head(10))
print("\n")

--- Vistazo a train_df (head) ---
   ID  id_season      aggregated_family   family  \
0   1         86  Dresses and jumpsuits  Dresses   
1   1         86  Dresses and jumpsuits  Dresses   
2   1         86  Dresses and jumpsuits  Dresses   
3   1         86  Dresses and jumpsuits  Dresses   
4   1         86  Dresses and jumpsuits  Dresses   

                              category fabric color_name  color_rgb  \
0  Dresses, jumpsuits and Complete set  WOVEN   AMARILLO  255,215,0   
1  Dresses, jumpsuits and Complete set  WOVEN   AMARILLO  255,215,0   
2  Dresses, jumpsuits and Complete set  WOVEN   AMARILLO  255,215,0   
3  Dresses, jumpsuits and Complete set  WOVEN   AMARILLO  255,215,0   
4  Dresses, jumpsuits and Complete set  WOVEN   AMARILLO  255,215,0   

                                     image_embedding length_type  \
0  0.072266474,-0.12752205,0.6080948,-1.2579741,-...        Long   
1  0.072266474,-0.12752205,0.6080948,-1.2579741,-...        Long   
2  0.072266474,-0.1275

In [53]:
# --- [INICIO] Código Corregido para Paso 2 ---

# --- Paso 2: Tarea 1 (Crear el Target 'y_train_agg') ---
print("Agregando 'weekly_demand' para crear el target (y_train_agg)...")

# (CORRECCIÓN): Usamos 'ID' (mayúscula) como vimos en tu CSV
y_train_agg = train_df.groupby('ID')['weekly_demand'].sum()

print("Target (y_train_agg) creado. Ejemplo:")
print(y_train_agg.head())
print("\n")


# --- Paso 2: Tarea 2 (Crear las Features 'X_train_agg') ---
print("Agregando features (X_train_agg)...")

# (CORRECCIÓN): Usamos 'ID' (mayúscula)
X_train_agg_full = train_df.groupby('ID').first()


# --- Tarea 2b: Alinear Columnas (¡NUEVA LÓGICA MÁS ROBUSTA!) ---
# En lugar de usar la lista 'test_df.columns' (que tiene 'ID' y 'Unnamed'),
# vamos a buscar las columnas que SÍ están en ambos sitios.

train_features = set(X_train_agg_full.columns)
test_features = set(test_df.columns)

# Buscamos la intersección: columnas que están en X_train_agg_full Y en test_df
# Esto excluye automáticamente 'ID' (que no es columna en train) 
# y 'Unnamed:' (que no están en train)
common_columns = list(train_features.intersection(test_features))

print(f"Encontradas {len(common_columns)} columnas en común.")

# Ahora filtramos ambos DataFrames para que SÓLO tengan estas columnas
X_train_agg = X_train_agg_full[common_columns].copy() # Usamos .copy() para evitar warnings

# ¡Importante! También limpiamos test_df para que coincida
# (Guardamos el 'ID' original de test_df para la sumisión final)
test_ids_for_submission = test_df['ID']
test_df_clean = test_df[common_columns].copy() 


# --- Tarea 2c: Alineación Final ---
# Nos aseguramos de que 'y_train_agg' siga el mismo orden que 'X_train_agg'
y_train_agg = y_train_agg.reindex(X_train_agg.index)

print("\n¡Agregación completada y alineada con éxito!")
print(f"Forma de X_train_agg: {X_train_agg.shape}")
print(f"Forma de y_train_agg: {y_train_agg.shape}")
print(f"Forma de test_df_clean: {test_df_clean.shape}")
print("\n")

print("--- Columnas de X_train_agg (.info()) ---")
X_train_agg.info()
print("\n")
print("--- Columnas de test_df_clean (.info()) ---")
test_df_clean.info()

# --- [FIN] Código Corregido para Paso 2 ---

Agregando 'weekly_demand' para crear el target (y_train_agg)...
Target (y_train_agg) creado. Ejemplo:
ID
1      806
2     2266
3    63791
4    11004
6    14684
Name: weekly_demand, dtype: int64


Agregando features (X_train_agg)...
Encontradas 27 columnas en común.

¡Agregación completada y alineada con éxito!
Forma de X_train_agg: (9843, 27)
Forma de y_train_agg: (9843,)
Forma de test_df_clean: (2250, 27)


--- Columnas de X_train_agg (.info()) ---
<class 'pandas.core.frame.DataFrame'>
Index: 9843 entries, 1 to 12767
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   moment              9843 non-null   object 
 1   archetype           5663 non-null   object 
 2   phase_in            9843 non-null   object 
 3   neck_lapel_type     6407 non-null   object 
 4   phase_out           9843 non-null   object 
 5   color_rgb           9843 non-null   object 
 6   print_type          9838 non-null   object 
 7

In [54]:
# --- Paso 3: Ingeniería de Características ---

print("Iniciando Paso 3: Ingeniería de Características...")

# Hacemos una copia para evitar 'SettingWithCopyWarning'
X_train_features = X_train_agg.copy()
X_test_features = test_df_clean.copy()

# --- Tarea 1: Crear 'start_month' (Ingeniería de Fechas) ---
# Convertimos 'phase_in' a formato fecha (MANDATORIO para extraer el mes)
# errors='coerce' convertirá cualquier fecha inválida en NaT (Nulo)
X_train_features['phase_in'] = pd.to_datetime(X_train_features['phase_in'], errors='coerce')
X_test_features['phase_in'] = pd.to_datetime(X_test_features['phase_in'], errors='coerce')

# Extraemos el mes (1-12) como una nueva feature numérica
X_train_features['start_month'] = X_train_features['phase_in'].dt.month
X_test_features['start_month'] = X_test_features['phase_in'].dt.month

# Ahora rellenamos cualquier nulo que se haya creado (por si 'errors=coerce' falló)
# Usaremos la mediana (ej. 6, para Junio/Julio)
median_month = X_train_features['start_month'].median()
X_train_features['start_month'] = X_train_features['start_month'].fillna(median_month)
X_test_features['start_month'] = X_test_features['start_month'].fillna(median_month)

print("Feature 'start_month' creada con éxito. Ejemplo:")
print(X_train_features[['phase_in', 'start_month']].head())
print("\n")


# --- Tarea 2: Definir Listas de Columnas para el Pipeline ---
# Basado en nuestro análisis del .info()

# 1. Columnas Numéricas: Rellenaremos nulos con la mediana y las escalaremos
numeric_features = [
    'num_stores',
    'price',
    'life_cycle_length',
    'num_sizes',
    'start_month'  # Nuestra nueva feature
]

# 2. Columnas Categóricas: Rellenaremos nulos con "Desconocido" y aplicaremos One-Hot Encoding
categorical_features = [
    'moment',
    'archetype',
    'neck_lapel_type',
    'print_type',
    'has_plus_sizes',       # Aunque es bool, lo tratamos como categórico
    'knit_structure',
    'waist_type',
    'silhouette_type',
    'family',
    'length_type',
    'color_name',
    'category',
    'woven_structure',
    'id_season',            # Aunque es int, es una CATEGORÍA (ej. temporada 86)
    'aggregated_family',
    'sleeve_length_type',
    'fabric'
]

# 3. Columnas a Eliminar: No las usaremos
# (El Pipeline se encargará de esto automáticamente al NO incluirlas 
# en las listas de arriba, excepto las que creamos y ya no necesitamos)

print(f"Definidas {len(numeric_features)} features numéricas:")
print(numeric_features)
print("\n")
print(f"Definidas {len(categorical_features)} features categóricas:")
print(categorical_features)
print("\n¡Paso 3 completado! Estamos listos para construir el Pipeline.")

Iniciando Paso 3: Ingeniería de Características...
Feature 'start_month' creada con éxito. Ejemplo:
     phase_in  start_month
ID                        
1  2023-02-01          2.0
2         NaT          6.0
3         NaT          6.0
4         NaT          6.0
6         NaT          6.0


Definidas 5 features numéricas:
['num_stores', 'price', 'life_cycle_length', 'num_sizes', 'start_month']


Definidas 17 features categóricas:
['moment', 'archetype', 'neck_lapel_type', 'print_type', 'has_plus_sizes', 'knit_structure', 'waist_type', 'silhouette_type', 'family', 'length_type', 'color_name', 'category', 'woven_structure', 'id_season', 'aggregated_family', 'sleeve_length_type', 'fabric']

¡Paso 3 completado! Estamos listos para construir el Pipeline.


In [55]:
# --- Paso 4: Preprocesamiento y Pipeline de Scikit-learn ---

# Importamos todas las herramientas que necesitamos
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

print("Herramientas de Scikit-learn importadas.")

# --- Tarea 1: Definir los Mini-Pipelines ---

# Pipeline para datos NUMÉRICOS
# 1. SimpleImputer: Rellena cualquier nulo (ej. en 'start_month') con la mediana.
# 2. StandardScaler: Escala los datos (pone todo ~ entre -2 y 2). 
#    RandomForest no lo necesita, pero es BUENA PRÁCTICA para otros modelos.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para datos CATEGÓRICOS
# 1. SimpleImputer: Rellena los nulos (ej. en 'archetype') con la palabra "Desconocido".
# 2. OneHotEncoder: Crea nuevas columnas para cada categoría (ej. family_Dresses, family_Coats).
#    handle_unknown='ignore' -> Si en test_df aparece un color que no vio en train,
#    simplemente lo ignora sin dar error. ¡Vital!
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

print("Pipelines de transformación numérica y categórica definidos.")

# --- Tarea 2: Unir con ColumnTransformer ---
# Aquí es donde le decimos a sklearn:
# - Aplica 'numeric_transformer' a las columnas de 'numeric_features'
# - Aplica 'categorical_transformer' a las columnas de 'categorical_features'
# - remainder='drop' -> Todas las columnas que no mencionamos (ej. image_embedding) serán ELIMINADAS.

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Ignora las columnas que no están en nuestras listas
)

print("ColumnTransformer ('preprocessor') creado con éxito.")
print("\n¡Paso 4 completado! El 'preprocessor' está listo para ser usado.")

# --- Bonus: Verifiquemos qué ha creado ---
# Vamos a "entrenar" solo el preprocesador y ver la forma de la salida
# Usamos las variables que creamos en el Paso 3

# Renombramos por simplicidad
X = X_train_features
y = y_train_agg
X_test = X_test_features

# 'fit_transform' aprende de X (ej. calcula la mediana) y luego transforma X
X_processed = preprocessor.fit_transform(X)

print("\n--- Verificación del Preprocesador ---")
print(f"Forma original de X_train: {X.shape}")
print(f"Forma de X_train procesado: {X_processed.shape}")
print("¡El número de columnas ha crecido por el OneHotEncoding!")

Herramientas de Scikit-learn importadas.
Pipelines de transformación numérica y categórica definidos.
ColumnTransformer ('preprocessor') creado con éxito.

¡Paso 4 completado! El 'preprocessor' está listo para ser usado.

--- Verificación del Preprocesador ---
Forma original de X_train: (9843, 28)
Forma de X_train procesado: (9843, 297)
¡El número de columnas ha crecido por el OneHotEncoding!


In [59]:
# --- Paso 5: Entrenamiento del Modelo ---
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

print("Herramientas de modelo y métricas importadas.")

# --- Tarea 1: Separar Datos de Validación ---
# Dividimos nuestros datos (X, y) en 80% para entrenar y 20% para validar
# random_state=42 asegura que la división sea siempre la misma
X_train_local, X_val_local, y_train_local, y_val_local = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Datos divididos: {len(X_train_local)} para entrenar, {len(X_val_local)} para validar.")
print("\n")

# --- Tarea 2: Crear el Pipeline Final ---
# Unimos el 'preprocessor' (Paso 4) y nuestro modelo 'RandomForestRegressor'

# n_estimators=100 -> Crear un bosque de 100 árboles (es un buen número para empezar)
# n_jobs=-1 -> Usar todos los núcleos de tu CPU para entrenar más rápido
# random_state=42 -> Para que el entrenamiento sea reproducible
model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

print("Pipeline final creado (Preprocesador + RandomForest).")

# --- Tarea 3: Entrenar ---
print("Entrenando el modelo... (Esto puede tardar 1-2 minutos)")
# ¡Aquí ocurre la magia!
# .fit() entrena el preprocesador y el modelo, todo en una línea.
full_pipeline.fit(X_train_local, y_train_local)

print("¡Modelo entrenado con éxito!")
print("\n")


# --- Paso 6: Evaluación y Ajuste ---

print("--- Evaluación del Modelo (sobre datos de validación) ---")

# 1. Predecir sobre los datos de validación
y_pred_val = full_pipeline.predict(X_val_local)

# 2. Calcular Métricas de Error
mae = mean_absolute_error(y_val_local, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val_local, y_pred_val))

print(f"Error Absoluto Medio (MAE): {mae:.4f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.4f}")
print("\n")

print("--- Interpretación ---")
print(f"Un MAE de {mae:.4f} significa que, en promedio, las predicciones de demanda total del modelo")
print(f"tienen un error de +/- {mae:.4f} unidades (en la escala 0-1).")
print("\n¡Paso 5 y 6 completados!")

Herramientas de modelo y métricas importadas.
Datos divididos: 7874 para entrenar, 1969 para validar.


Pipeline final creado (Preprocesador + RandomForest).
Entrenando el modelo... (Esto puede tardar 1-2 minutos)
¡Modelo entrenado con éxito!


--- Evaluación del Modelo (sobre datos de validación) ---
Error Absoluto Medio (MAE): 3822.0330
Raíz del Error Cuadrático Medio (RMSE): 7800.2950


--- Interpretación ---
Un MAE de 3822.0330 significa que, en promedio, las predicciones de demanda total del modelo
tienen un error de +/- 3822.0330 unidades (en la escala 0-1).

¡Paso 5 y 6 completados!


In [57]:
# --- Paso 7: Predicción Final y Archivo de Envío ---

print("Iniciando Paso 7: Predicción Final...")

# --- Tarea 1: Entrenar con TODOS los datos ---
print("Re-entrenando el modelo con el 100% de los datos de entrenamiento...")
# Usamos 'X' e 'y' (los DataFrames completos que creamos en el Paso 3)
# 'X_test_features' es el set de test limpio
full_pipeline.fit(X, y)

print("¡Modelo final entrenado!")

# --- Tarea 2: Predecir sobre test_df_clean ---
print("Generando predicciones sobre los datos de test...")
# (Recuerda que X_test_features es el nombre que le dimos a test_df_clean en el Paso 3)
final_predictions = full_pipeline.predict(X_test_features)

print("¡Predicciones generadas!")

# --- Tarea 3: Post-Procesamiento (¡MUY IMPORTANTE!) ---
# La demanda no puede ser negativa.
# Reemplazamos cualquier predicción negativa por 0.
final_predictions[final_predictions < 0] = 0
print("Predicciones negativas ajustadas a 0.")

# --- Tarea 4: Crear Archivo de Envío ---
# Usamos los 'test_ids_for_submission' que guardamos en el Paso 2
# y nuestras 'final_predictions'
submission_df = pd.DataFrame({
    'ID': test_ids_for_submission,
    'demand': final_predictions
})

# --- Tarea 5: Guardar el Archivo ---
submission_filename = 'submission2.csv'
submission_df.to_csv(submission_filename, index=False, sep=',')

print("\n")
print("="*50)
print(f"¡Archivo '{submission_filename}' creado con éxito!")
print("="*50)
print("Vistazo al archivo de envío:")
print(submission_df.head())
print("\n")
print("¡Paso 7 completado!")

Iniciando Paso 7: Predicción Final...
Re-entrenando el modelo con el 100% de los datos de entrenamiento...
¡Modelo final entrenado!
Generando predicciones sobre los datos de test...
¡Predicciones generadas!
Predicciones negativas ajustadas a 0.


¡Archivo 'submission2.csv' creado con éxito!
Vistazo al archivo de envío:
    ID   demand
0   90  1272.69
1   16  6059.90
2   65  9059.40
3  138   820.80
4  166   328.27


¡Paso 7 completado!
